In [1]:
from toolz.curried import *
from whatever4e import Chain, Whatever
import pandas as pd
import sklearn
from sklearn import *
from inspect import isclass

/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [2]:
seconds = 2

In [3]:
def scrape_models(module, base=sklearn.base.ClassifierMixin):
    module = getattr(sklearn, module)
    return Chain(module) | dir | map(partial(partial(getattr, module))) \
        | filter(isclass) | filter(partial(flip(issubclass), base)) \
        > list

In [4]:
all_of_the_classifier_models = Chain(sklearn) | dir \
    | map(scrape_models) | concat | unique > list

## Training our model

Create some testing and validation data from the `digits` dataset.

In [38]:
X, y = Chain(sklearn.datasets.load_digits()) > get(['data', 'target']) 
Xt, Xv, yt, yv = sklearn.cross_validation.train_test_split(X, y)

#### Throttle the model with `stopit`

> Be mindful of copypasta

In [39]:
from stopit import ThreadingTimeout
@curry
def time_and_catch_model(seconds, m, X=Xt, y=yt, attr='fit'):
    """Execute and model fit, prediction, score or transform
    method that takes less than `seconds`.
    """
    with ThreadingTimeout(seconds) as t:
        try: 
            if attr in ['fit']:
                m = m()
            if attr in ['predict', 'transform']:
                return getattr(m, attr)(X)
            return getattr(m, attr)(X, y)
        except:
            return False
    return False            

In [40]:
all_the_fit_models = Chain(all_of_the_classifier_models) | map(
    time_and_catch_model(seconds)
) | filter(bool) > list

/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/covariance/robust_covariance.py:613: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:688: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [41]:
predicted = Chain(all_the_fit_models) | map(
    partial(time_and_catch_model(seconds), X=X, y=y, attr='predict')
) > list

In [42]:
prediction_frame = Chain(predicted).zip(all_the_fit_models).dict() \
    | valfilter(complement(
            partial(flip(isinstance), (bool))
        )) \
    | keymap(lambda x: first(str(x).split('('))) \
    > partial(pd.DataFrame, index=y)
prediction_frame

,AdaBoostClassifier,BaggingClassifier,BernoulliNB,CalibratedClassifierCV,DecisionTreeClassifier,DummyClassifier,ExtraTreeClassifier,ExtraTreesClassifier,GaussianNB,KNeighborsClassifier,...,NearestCentroid,NuSVC,PassiveAggressiveClassifier,Perceptron,QuadraticDiscriminantAnalysis,RandomForestClassifier,RidgeClassifier,RidgeClassifierCV,SGDClassifier,SVC
0,0,0,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,9,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,2,1,2,2,9,2,2,8,2,...,1,2,2,2,2,2,2,2,2,2
3,1,3,3,3,3,4,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,0,4,4,4,4,3,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
5,1,5,3,5,5,6,5,5,3,9,...,9,5,9,9,5,5,9,9,9,5
6,6,6,6,6,6,8,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
7,1,7,7,7,4,3,7,7,7,7,...,7,9,7,7,7,7,7,7,7,9
8,1,8,8,8,8,5,8,8,8,8,...,8,9,8,8,8,8,8,8,8,9
9,1,9,9,9,9,7,9,9,8,9,...,9,9,9,9,9,9,9,9,9,9


Sort the indices by the score

In [43]:
scores = Chain(all_the_fit_models) | map(
        partial(time_and_catch_model(seconds), X=X, y=y, attr='score')
    ) | partial(zip, all_the_fit_models) | dict \
    | keymap(lambda x: first(str(x).split('('))) \
    > pd.Series

In [44]:
df = prediction_frame[
        Chain(scores.sort_values(ascending=False).index) \
            | filter(lambda x: x in prediction_frame.columns) \
            > list
    ]

In [45]:
%matplotlib inline
import matplotlib.pyplot as plt

In [47]:
from sklearn.pipeline import make_union

In [60]:
g = transformers[0]

In [54]:
transformers = Chain(all_the_fit_models) | filter(partial(flip(hasattr), 'transform')) \
    map

/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is

In [58]:
v.shape

(1797, 262)

In [46]:
Chain(all_the_fit_models) | filter(partial(flip(hasattr), 'transform')) \
    | map(partial(time_and_catch_model(2), attr='transform')) > list

/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/tfast/anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is

[array([[  0.,   4.,  16., ...,   4.,  15.,   4.],
        [  0.,   4.,  12., ...,   5.,  16.,   7.],
        [  0.,   1.,  13., ...,   0.,   8.,   0.],
        ..., 
        [  0.,   7.,  13., ...,   5.,  12.,  12.],
        [  0.,  14.,  14., ...,  13.,  10.,   1.],
        [  1.,  14.,  16., ...,  15.,   0.,   0.]]),
 array([[ 3.27434422,  1.99164079,  2.57735168, ...,  1.12367786,
          0.38959927, -0.30922218],
        [ 2.26989355, -1.72410205, -0.18458119, ..., -0.47925973,
          1.52694302, -2.99060235],
        [-2.61759104, -4.7300878 ,  4.663134  , ...,  1.50774968,
         -2.78636647,  0.18211057],
        ..., 
        [ 3.82673979,  3.46323769,  1.13698039, ...,  0.20911402,
         -0.06044534, -0.13826642],
        [ 4.34829764, -1.08988114,  1.24915299, ..., -2.82157334,
         -2.8689322 ,  0.60685143],
        [ 0.50300376, -2.58709142, -2.50170326, ..., -0.1045039 ,
         -0.27683614,  0.03635004]]),
 array([[  4.,  16.,   6., ...,   1.,  16.,  15.],

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x = y = pd.np.arange(df.shape[1])

c = ax.pcolor(*pd.np.meshgrid(x, y), df.corr().fillna(0).values)
plt.colorbar(c);

In [14]:
prediction_frame.corr()

,AdaBoostClassifier,BaggingClassifier,BernoulliNB,CalibratedClassifierCV,DecisionTreeClassifier,DummyClassifier,ExtraTreeClassifier,ExtraTreesClassifier,GaussianNB,KNeighborsClassifier,...,NearestCentroid,NuSVC,PassiveAggressiveClassifier,Perceptron,QuadraticDiscriminantAnalysis,RandomForestClassifier,RidgeClassifier,RidgeClassifierCV,SGDClassifier,SVC
AdaBoostClassifier,1.000000,0.386335,0.374160,0.398982,0.411471,0.113607,0.280558,0.392435,0.323222,0.398664,...,0.394226,0.341411,0.388597,0.377304,0.387773,0.386535,0.372216,0.372216,0.369233,0.333817
BaggingClassifier,0.386335,1.000000,0.756539,0.902474,0.805164,0.052670,0.633137,0.902667,0.796748,0.934957,...,0.827497,0.684694,0.884897,0.863592,0.826358,0.901436,0.889598,0.889598,0.786186,0.648725
BernoulliNB,0.374160,0.756539,1.000000,0.791309,0.702550,0.097369,0.571253,0.777355,0.771438,0.768438,...,0.801792,0.598327,0.796943,0.765850,0.728682,0.774504,0.787176,0.787176,0.795562,0.583007
CalibratedClassifierCV,0.398982,0.902474,0.791309,1.000000,0.805513,0.046667,0.630747,0.936286,0.802744,0.950550,...,0.875397,0.679015,0.931928,0.923391,0.850441,0.940492,0.939581,0.939581,0.815868,0.642554
DecisionTreeClassifier,0.411471,0.805164,0.702550,0.805513,1.000000,0.032282,0.572158,0.799800,0.721005,0.814857,...,0.762580,0.622661,0.813916,0.779438,0.764216,0.803588,0.796498,0.796498,0.726855,0.589279
DummyClassifier,0.113607,0.052670,0.097369,0.046667,0.032282,1.000000,0.006589,0.076707,0.077188,0.055966,...,0.063311,0.023832,0.030787,0.034187,0.043822,0.054143,0.046559,0.046559,0.083780,0.029411
ExtraTreeClassifier,0.280558,0.633137,0.571253,0.630747,0.572158,0.006589,1.000000,0.648955,0.573641,0.648516,...,0.578936,0.559759,0.640465,0.640592,0.605674,0.646737,0.661038,0.661038,0.553588,0.545473
ExtraTreesClassifier,0.392435,0.902667,0.777355,0.936286,0.799800,0.076707,0.648955,1.000000,0.795810,0.971361,...,0.853996,0.685093,0.907489,0.913414,0.874228,0.958354,0.908443,0.908443,0.802536,0.649185
GaussianNB,0.323222,0.796748,0.771438,0.802744,0.721005,0.077188,0.573641,0.795810,1.000000,0.805089,...,0.801446,0.614205,0.787393,0.770688,0.767566,0.783926,0.810979,0.810979,0.767030,0.578438
KNeighborsClassifier,0.398664,0.934957,0.768438,0.950550,0.814857,0.055966,0.648516,0.971361,0.805089,1.000000,...,0.844984,0.692119,0.918209,0.914155,0.885178,0.955118,0.923688,0.923688,0.808484,0.656058
